#### Coursera - Capstone Project

In [37]:
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup
import requests
import lxml.html as lh

In [149]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library


In [2]:
print("Hello Capstome Project Course!")

Hello Capstome Project Course!


## Capstone Project Week 3 :
#### Segmenting and Clustering Neighborhoods in Toronto

In [53]:
wikiUrl =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [61]:
df=pd.read_html(wikiUrl, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [68]:
#  Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']

In [95]:
dfGroup = df.groupby(['Postcode', 'Borough']).Neighbourhood.agg(", ".join).reset_index()

In [96]:
# check group df 
dfGroup[dfGroup.Postcode == 'M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [112]:
# for each row in dfGroup call replaceValue function and check if Neighbourhood is 'Not assigned'
dfGroup['Neighbourhood'] = dfGroup.apply(replaceValue, axis=1)

In [136]:
dfGroup[dfGroup.Neighbourhood == "Queen's Park"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [107]:
# function: takes single rows and checks if Neighbourhood value is 'Not assigned'
def replaceValue(row):
    ''' Check if neighbourhood is Not assigned and replace with Borough'''
    if (row['Neighbourhood'] == 'Not assigned'):
        return (row['Borough'])
    else:
        return (row['Neighbourhood'])

In [111]:
dfGroup.shape

(103, 3)

In [122]:
df.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

### Geospatial coding

In [114]:
dfLatLong = pd.read_csv('Geospatial_Coordinates.csv')

In [118]:
dfLatLong.rename(columns={"Postal Code":'Postcode'}, inplace=True)

In [119]:
dfLatLong.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [141]:
dfFinal = pd.merge(dfGroup, dfLatLong, on="Postcode")

# Explore and cluster the neighborhoods in Toronto.

In [142]:
toronto_data = dfFinal[dfFinal.Borough == 'Downtown Toronto'].reset_index(drop=True)

In [143]:
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [147]:
address = 'Downtown Toronto, Canada'

geolocator = Nominatim(user_agent="torontoExplorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [151]:
# create map of Manhattan using latitude and longitude values
mapToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapToronto)  
    
mapToronto